# Import Libraries

In [1]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords

# Read 100 questions list

In [2]:
df = pd.read_excel('AnswersFromSource11_20.xlsx',engine='openpyxl')
link = 'https://www.emedicinehealth.com/autism_spectrum_disorder_quiz_iq/faq.htm'
df[link]=np.nan
#df.drop(df.iloc[:, 1:25], inplace = True, axis = 1)
df

,Question,https://springbrookbehavioral.com/frequently-asked-questions-about-autism/,https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx,https://www.autism.org.sg/living-with-autism/myths-and-facts-of-autism,https://www.autismsa.org.au/frequently-asked-questions-about-asd,https://www.bbrfoundation.org/faq/frequently-asked-questions-about-autism,https://www.cdc.gov/ncbddd/autism/topics.html,https://www.emedicinehealth.com/autism_spectrum_disorder_quiz_iq/faq.htm
0,What Can I Do To Prepare My Child For The Future?,Early intervention can do a lot toward fosteri...,NaN,NaN,NaN,NaN,NaN,NaN
1,Did I Cause My Child’s Autism?,"No. After getting an ASD diagnosis, this is th...",NaN,NaN,NaN,NaN,NaN,NaN
2,Can My Child’s Autism Be Cured?,No. Autism is not a disease that can be cured—...,NaN,NaN,NaN,NaN,NaN,NaN
3,How Can I Deal With This Diagnosis?,"Start by being patient and kind with yourself,...",NaN,NaN,NaN,NaN,NaN,NaN
4,How Do I Find Help For My Child?,Start by making sure that you and your child h...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
130,What are some ways that parents can reduce the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,Do some families deal with stress better than ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,Do siblings suffer increased stress as a resul...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,What can I do about my children’s stress?,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Scrape QA pairs from website

In [3]:
# Open webpage in a new window
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.emedicinehealth.com/autism_spectrum_disorder_quiz_iq/faq.htm')



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/home/gopan/.wdm/drivers/chromedriver/linux64/92.0.4515.107/chromedriver] found in cache


In [4]:
# Parse text in webpage
source = driver.page_source
soup = bs4.BeautifulSoup(source, 'html.parser')

In [6]:
# Search the questions mentioned in webpage
l = []
for h4 in soup.find_all('h4',{'class':'faqHead'}):
    for span in h4.find_all('span',{'class':'ques'}):
        print(span.text)
        l.append(span.text)

Autism is a neuropsychiatric disorder. True or false?
Who is more likely to develop autism? Boys or girls?
Autism is almost always genetic. True or false?
Asperger syndrome is also on the spectrum of autism disorders. True or false?
There are no treatments for autism. True or false?
Autism includes a broad spectrum of symptoms and disorders. True or false?
Mothers were originally blamed for their child's autism. True or false?
Symptoms of autism are usually the same in every child. True or false?


In [8]:
# Tried to search for answers of questions
k=[]
c = 0
for i in soup.find_all('span',{'class':'ans'}):
    txt = i.text.strip()
    if(txt in l):
        print('*'*100)
    if(txt in l or len(txt.split())>1):
        print(c,'-->', txt)
    c = c+1
    
    k.append(txt)

0 --> True.


Autism spectrum disorder (ASD), often referred to as simply autism, is a developmental disability that impacts how people communicate verbally and nonverbally, and how they interact, behave, and learn. People with ASD don't look different, but the way they interact with the world around them is different. The way people with ASD learn, think, and problem solve can vary from being gifted in math, art, music, or visual skills, to being severely challenged both mentally and physically. Many people with ASD can live independently, while some need assistance in daily living.
1 --> Boys.


Boys are diagnosed with autism four times more often than girls. One in 68 children in the U.S. is diagnosed with ASD, and this spans all races, regions, and socio-economic statuses. 


Approximately 40% of those diagnosed with autism are non-verbal. About 25%-30% of children with autism may speak some words around age 12 to 18 months, but then lose them. Others may gain the ability to speak 

# Check which questions are similar

In [9]:
stop_words = set(stopwords.words('english'))

def clean_text(sent):
    sent = sent.lower() # lowercase
    sent = re.sub(r'[^\w\s]', '', sent) # remove punctuations
    sent = re.sub('Autism Spectrum Disorder','ASD',sent) # Compress term
    sent = [w for w in sent.split() if not w.lower() in stop_words] # Remove stopwords
    sent = " ".join(sent)
    return sent

In [10]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [12]:
for i in l:
    i2 = clean_text(i)
    list1 = i2.split()
    
    temp1 = 0.0
    temp2 = ''
    temp3 = ''
    count = 0
    for z1, j in enumerate(df['Question']):
        j2 = clean_text(j)
        list2 = j2.split()
            
        sim = jaccard_similarity(list1, list2)
        
        if(sim>temp1):
            temp1 = sim
            temp2 = j
            temp3 = i
            count = z1+1
    if(temp1>=0.3):
        print('Website --> ',temp3)
        print('100 questions[ ',count,' ] list --> ',temp2)
        print(temp1)
        print('*'*100)

Website -->  Asperger syndrome is also on the spectrum of autism disorders. True or false?
100 questions[  27  ] list -->  What are the Autism Spectrum Disorders (ASD)?
0.3333333333333333
****************************************************************************************************
Website -->  There are no treatments for autism. True or false?
100 questions[  14  ] list -->  Are there medication treatments for autism?
0.4
****************************************************************************************************
Website -->  Autism includes a broad spectrum of symptoms and disorders. True or false?
100 questions[  27  ] list -->  What are the Autism Spectrum Disorders (ASD)?
0.3333333333333333
****************************************************************************************************


In [42]:
# Directly add answer to 'What are the Autism Spectrum Disorders (ASD)?' in existing dataframe
#df.iloc[2,0]
#df.iloc[1,1]=k[3]

In [13]:

l

['Autism is a neuropsychiatric disorder. True or false?',
 'Who is more likely to develop autism? Boys or girls?',
 'Autism is almost always genetic. True or false?',
 'Asperger syndrome is also on the spectrum of autism disorders. True or false?',
 'There are no treatments for autism. True or false?',
 'Autism includes a broad spectrum of symptoms and disorders. True or false?',
 "Mothers were originally blamed for their child's autism. True or false?",
 'Symptoms of autism are usually the same in every child. True or false?']

In [44]:
#l=l[1:]
#l

In [22]:
# manually select answers to questions in list 'l'

#l2 = [k[8]+'\n'+k[12]+'\n'+k[13]+'\n'+k[14],
#      k[16]+'\n'+k[17],
#     k[19]+'\n'+k[21]]
l2 = k

In [23]:
# Create new dataframe with QA pairs
df2 = pd.DataFrame(zip(l,l2),columns=['Question',link])
df2

,Question,https://www.emedicinehealth.com/autism_spectrum_disorder_quiz_iq/faq.htm
0,Autism is a neuropsychiatric disorder. True or...,"True.\n\n\nAutism spectrum disorder (ASD), oft..."
1,Who is more likely to develop autism? Boys or ...,Boys.\n\n\nBoys are diagnosed with autism four...
2,Autism is almost always genetic. True or false?,False.\n\n\nThe National Institutes of Health ...
3,Asperger syndrome is also on the spectrum of a...,True.\n\n\nA condition also on the spectrum of...
4,There are no treatments for autism. True or fa...,False.\n\n\nWhile there are no treatments to c...
5,Autism includes a broad spectrum of symptoms a...,"True.\n\n\nThe term ""autism"" is generally used..."
6,Mothers were originally blamed for their child...,True.\n\n\nIn the past mothers were blamed for...
7,Symptoms of autism are usually the same in eve...,False. \n\n \nSymptoms of autism vary from chi...


In [24]:
# Concatenate existing and new dataframes
df3 = pd.concat([df,df2],axis=0)
df3 = df3.sort_values(by=list(df3.columns[1:])).reset_index(drop=True)
df3

,Question,https://springbrookbehavioral.com/frequently-asked-questions-about-autism/,https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx,https://www.autism.org.sg/living-with-autism/myths-and-facts-of-autism,https://www.autismsa.org.au/frequently-asked-questions-about-asd,https://www.bbrfoundation.org/faq/frequently-asked-questions-about-autism,https://www.cdc.gov/ncbddd/autism/topics.html,https://www.emedicinehealth.com/autism_spectrum_disorder_quiz_iq/faq.htm
0,What Can I Do To Prepare My Child For The Future?,Early intervention can do a lot toward fosteri...,NaN,NaN,NaN,NaN,NaN,NaN
1,Did I Cause My Child’s Autism?,"No. After getting an ASD diagnosis, this is th...",NaN,NaN,NaN,NaN,NaN,NaN
2,Can My Child’s Autism Be Cured?,No. Autism is not a disease that can be cured—...,NaN,NaN,NaN,NaN,NaN,NaN
3,How Can I Deal With This Diagnosis?,"Start by being patient and kind with yourself,...",NaN,NaN,NaN,NaN,NaN,NaN
4,How Do I Find Help For My Child?,Start by making sure that you and your child h...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
138,What are some ways that parents can reduce the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,Do some families deal with stress better than ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,Do siblings suffer increased stress as a resul...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,What can I do about my children’s stress?,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Save Output

In [25]:
df3.to_excel('AnswersFromSource11_20.xlsx',index=False)